In [19]:
import requests, praw, re, time
from bs4 import BeautifulSoup 
from datetime import datetime
import pandas as pd 
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from config import indeed_query,r_client_id,r_secret_key,r_dev,r_username,r_pw

In [20]:
reddit = praw.Reddit(
    client_id=r_client_id,
    client_secret=r_secret_key,
    password=r_pw,
    user_agent=f'testscript by u/{r_dev}',
    username=r_dev,
)
print(reddit.user.me())

Jazzy_Danger_1230


In [29]:
submissions=reddit.subreddit("wallstreetbets").search("Daily Discussion Thread", limit=4, sort='new')

In [ ]:
lst = []
for submission in submissions:
    #print(submission.title)    
    submission.comments.replace_more(limit=None)
    submission.comment_sort = 'new'
    for comment in submission.comments: 
        try:
            lst.append(
                {
                    'submission_title':submission.title
                    ,'comment_body':comment.body
                    ,'time':datetime.utcfromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S')
                    ,'author': comment.author
                    ,'score':comment.score
                    ,'permalink':comment.permalink
                    ,'replies':comment.replies
                    ,'id':comment.id 
                }
            ) 
        except:
            print('err' + comment.id  )
            pass    
df = pd.DataFrame(lst)
df

In [13]:
for submission in reddit.subreddit("wallstreetbets").hot(limit=1):
    print(submission.title)
    submission.comment_sort = 'new'
    for comment in submission.comments:
        try:
            if 'BB' in comment.body:
                print(comment.body)
        except:
            pass

Daily Discussion Thread for June 02, 2021
BB LETS GO 🚀 🚀 🚀
BB is waiting for takeoff
idk when the amc run will end, but when it does, buy calls on BB
BB IS THE WAY
You can’t BBeat us down!!!
GEORGE DUBBYA ON AMC
Grab your BB calls today before be to late . Today  BB will start taking off 🚀🛸
Shifting profits to BB. Let's go!
MELVIN SHORT LADDERING $BB !!!

TIME TO GET INTO $BB!!
Hop in BB while AMC recharges
What's up with all this BB bullshit in here??
BB bout to moon 🚀🚀
Halt just means you buy more BB while waiting. No worries here.
I JUST BOUGHT 10K OF BB BECAUSE FOMO. APES, DO NOT LET ME DOWN AND HOLLDDDDDDD
Nervous about BB it’s moving with AMC so whatever AMC does is what BB will do. Hopefully not down because I’m betting on BB being next
When will BB take off? Leaps are cheap. Just buy them and it’ll work out.
How high will BB fly?
HEY NEW MONEY AMC APES, COME ON OVER TO BB. THE WATER IS NICE.
 ANOTHER AMC HALT....  I'M BUYING MORE BB.
BB is so strong on its own
I havent got much

In [ ]:
p=0
url = f'https://washingtondc.craigslist.org/d/bicycles/search/bia?s={p}'
soup = BeautifulSoup(requests.get(url).text, "html.parser")

   
    